In [1]:
# criar um navegador
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

import pandas as pd

# criar o navegador
driver = webdriver.Chrome()

# importar/visualizar a base de dados
tabela_produtos = pd.read_excel(r"C:\Users\alves\OneDrive\Projeto selenium\buscas.xlsx")
display(tabela_produtos)

,Nome,Termos banidos,Preço mínimo,Preço máximo
0,iphone 12 64gb,mini watch,3000,3500
1,rtx 3060,zota galax,2000,3500


Definição das funções de busca no google e no buscape

In [2]:
import time
def busca_google_shopping(driver, produto, termos_banidos, preco_minimo, preco_maximo):
    #Entrando no google    
    driver.get("https://www.google.com")

    # tratar os valores que vieram da tabela
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(" ")
    lista_termos_produto = produto.split(" ")
    preco_minimo = float(preco_minimo)   
    preco_maximo = float(preco_maximo)
    # pesquisar o nome do produto no google
    driver.find_element(By.XPATH, '//*[@id="APjFqb"]').send_keys(produto)
    driver.find_element(By.XPATH, '//*[@id="APjFqb"]').send_keys(Keys.ENTER)

    # clicar na aba shopping
    time.sleep(1)
    driver.find_element(By.XPATH,'//*[@id="cnt"]/div[5]/div/div/div/div[1]/div/a[1]').click()

    # pegar a lista de resultados da busca no google shopping
    time.sleep(1)
    driver.execute_script(f"window.scroll(0, 1350);")
    time.sleep(1)
    lista_resultados = driver.find_elements(By.CLASS_NAME, 'sh-dgr__gr-auto')

    # para cada resultado, ele vai verificar se o resultado corresponde a todas as nossas condicoes
    lista_ofertas = [] # lista que a função vai me dar como resposta
    for resultado in lista_resultados:
        nome = resultado.find_element(By.CLASS_NAME, 'tAxDx').text
        nome = nome.lower()
        # verificacao do nome - se no nome tem algum termo banido
        tem_termos_banidos = False
        for palavra in lista_termos_banidos:
            if palavra in nome:
                tem_termos_banidos = True
        # verificar se no nome tem todos os termos do nome do produto
        tem_todos_termos_produto = True
        for palavra in lista_termos_produto:
            if palavra not in nome:
                tem_todos_termos_produto = False

        if not tem_termos_banidos and tem_todos_termos_produto:# verificando o nome
            try:
                preco = resultado.find_element(By.CLASS_NAME, 'a8Pemb').text
                preco = preco.replace("R$", "").replace(" ", "").replace(".", "").replace(",", ".")
                if preco==' ':
                    pass
                else:
                    preco = float(preco)
                # verificando se o preco ta dentro do minimo e maximo
                if preco_minimo <= preco <= preco_maximo:
                    elemento_link = resultado.find_element(By.CLASS_NAME, 'aULzUe')
                    elemento_pai = elemento_link.find_element(By.XPATH, '..')
                    link = elemento_pai.get_attribute('href')
                    lista_ofertas.append((nome, preco, link))
            except:
                continue
    return lista_ofertas


def busca_buscape(nav, produto, termos_banidos, preco_minimo, preco_maximo):
    
    # tratar os valores da função
    preco_maximo = float(preco_maximo)
    preco_minimo = float(preco_minimo)
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(" ")
    lista_termos_produto = produto.split(" ")
    
    
    # entrar no buscape
    nav.get("https://www.buscape.com.br/")
    
    # pesquisar pelo produto no buscape
    nav.find_element(By.CLASS_NAME, 'AutoCompleteStyle_input__HG105').send_keys(produto, Keys.ENTER)
    
    # pegar a lista de resultados da busca do buscape
    time.sleep(5)
    nav.execute_script(f"window.scroll(0, 1350);")
    time.sleep(1)
    lista_resultados = nav.find_elements(By.CLASS_NAME, 'Paper_Paper__HIHv0')
    
    # para cada resultado
    lista_ofertas = []
    for resultado in lista_resultados:
        try:
            preco = resultado.find_element(By.CLASS_NAME, 'Text_MobileHeadingS__Zxam2').text
            nome = resultado.find_element(By.CLASS_NAME,'SearchCard_ProductCard_NameWrapper__Gv0x_').text
            nome = nome.lower()
            elemento_link = resultado.find_element(By.CLASS_NAME,'SearchCard_ProductCard_SuperiorTags__Ua2qE')
            elemento_pai = elemento_link.find_element(By.XPATH, '..')
            link = elemento_pai.get_attribute('href')
            
            # verificacao do nome - se no nome tem algum termo banido
            tem_termos_banidos = False
            for palavra in lista_termos_banidos:
                if palavra in nome:
                    tem_termos_banidos = True  
                    
            # verificar se no nome tem todos os termos do nome do produto
            tem_todos_termos_produto = True
            for palavra in lista_termos_produto:
                if palavra not in nome:
                    tem_todos_termos_produto = False            
            
            if not tem_termos_banidos and tem_todos_termos_produto:
                preco = preco.replace("R$", "").replace(" ", "").replace(".", "").replace(",", ".")
                preco = float(preco)
                if preco_minimo <= preco <= preco_maximo:
                    lista_ofertas.append((nome, preco, link))
        except:
            continue
    return lista_ofertas

In [3]:
busca_google_shopping(driver,'iphone 12 64gb','mini watch',3000,3500)

[('vitrine: iphone 12 apple 64gb roxo tela 6,1 câmera traseira 12mp ios',
  3134.05,
  'https://www.google.com/url?url=https://www.carrefour.com.br/vitrine-iphone-12-apple-64gb-roxo-tela-61-camera-traseira-12mp-ios-mp932337971/p&rct=j&q=&esrc=s&sa=U&ved=0ahUKEwit8c2OsPn_AhWWvJUCHZ9SA9sQgOUECLgM&usg=AOvVaw1TiPMZ-LaqEMBUsG6E8JbX'),
 ('vitrine: iphone 12 apple 64gb branco tela 6,1 câmera traseira 12mp',
  3134.05,
  'https://www.google.com/url?url=https://www.carrefour.com.br/vitrine-iphone-12-apple-64gb-branco-tela-61-camera-traseira-12mp-mp932985542/p&rct=j&q=&esrc=s&sa=U&ved=0ahUKEwit8c2OsPn_AhWWvJUCHZ9SA9sQgOUECMgM&usg=AOvVaw1wkt8OpudD_yMcTo9KIN3F'),
 ('iphone 12 64gb preto de vitrine tela 6,1 4g câmera traseira 12mp 12mp vitrine',
  3349.0,
  'https://www.google.com/url?url=https://www.carrefour.com.br/iphone-12-64gb-preto-de-vitrine-tela-61-4g-camera-traseira-12mp-12mp-vitrine-mp932568692/p&rct=j&q=&esrc=s&sa=U&ved=0ahUKEwit8c2OsPn_AhWWvJUCHZ9SA9sQgOUECPsM&usg=AOvVaw1aoxrvmEkbkKeNGh

In [4]:
busca_buscape(driver, 'rtx 3060', 'zota galax', 2000, 3500)

[('placa de video nvidia geforce rtx 3060 ti 8 gb gddr6 256 bits gainward ne6306t019p2-190ab v1',
  2399.9,
  'https://www.buscape.com.br/placa-de-video/placa-de-video-nvidia-geforce-rtx-3060-ti-8-gb-gddr6-256-bits-gainward-ne6306t019p2-190ab-v1?_lc=88&searchterm=rtx%203060'),
 ('placa de video nvidia geforce rtx 3060 12 gb gddr6 192 bits gainward ne63060019k9-190au',
  2080.0,
  'https://www.buscape.com.br/placa-de-video/placa-de-video-nvidia-geforce-rtx-3060-12-gb-gddr6-192-bits-gainward-ne63060019k9-190au?_lc=88&searchterm=rtx%203060'),
 ('placa de video nvidia geforce rtx 3060 12 gb gddr6 192 bits palit ne63060019k9-190ad',
  2599.0,
  'https://www.buscape.com.br/placa-de-video/placa-de-video-nvidia-geforce-rtx-3060-12-gb-gddr6-192-bits-palit-ne63060019k9-190ad?_lc=88&searchterm=rtx%203060'),
 ('placa de video rtx 3060 ti dual fan pny nvidia geforce, 8 gb gddr6, lhr, dlss, ray tracing - vcg3060t8ldfbpb1',
  2199.99,
  'https://www.buscape.com.br/lead?oid=942617381&channel=86&index=

Contrução da nossa lista de ofertas encontradas

In [5]:
tabela_ofertas = pd.DataFrame()

for linha in tabela_produtos.index:
    produto = tabela_produtos.loc[linha, "Nome"]
    termos_banidos = tabela_produtos.loc[linha, "Termos banidos"]
    preco_minimo = tabela_produtos.loc[linha, "Preço mínimo"]
    preco_maximo = tabela_produtos.loc[linha, "Preço máximo"]
    
    lista_ofertas_google_shopping = busca_google_shopping(driver, produto, termos_banidos, preco_minimo, preco_maximo)
    if lista_ofertas_google_shopping:
        tabela_google_shopping = pd.DataFrame(lista_ofertas_google_shopping, columns=['produto', 'preco', 'link'])
        tabela_ofertas = tabela_ofertas.append(tabela_google_shopping)
    else:
        tabela_google_shopping = None
        
    lista_ofertas_buscape = busca_buscape(driver, produto, termos_banidos, preco_minimo, preco_maximo)
    if lista_ofertas_buscape:
        tabela_buscape = pd.DataFrame(lista_ofertas_buscape, columns=['produto', 'preco', 'link'])
        tabela_ofertas = tabela_ofertas.append(tabela_buscape)
    else:
        tabela_buscape = None
        
#Colocando os índicies em forma sequencial        
tabela_ofertas.reset_index(inplace=True, drop=True)
display(tabela_ofertas)

C:\Users\alves\AppData\Local\Temp\ipykernel_5308\762012866.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabela_ofertas = tabela_ofertas.append(tabela_google_shopping)
C:\Users\alves\AppData\Local\Temp\ipykernel_5308\762012866.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabela_ofertas = tabela_ofertas.append(tabela_buscape)
C:\Users\alves\AppData\Local\Temp\ipykernel_5308\762012866.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabela_ofertas = tabela_ofertas.append(tabela_google_shopping)
C:\Users\alves\AppData\Local\Temp\ipykernel_5308\762012866.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabela_ofertas 

,produto,preco,link
0,"vitrine: iphone 12 apple 64gb roxo tela 6,1 câ...",3134.05,https://www.google.com/url?url=https://www.car...
1,"vitrine: iphone 12 apple 64gb branco tela 6,1 ...",3134.05,https://www.google.com/url?url=https://www.car...
2,"iphone 12 64gb preto de vitrine tela 6,1 4g câ...",3349.00,https://www.google.com/url?url=https://www.car...
3,"vitrine: iphone 12 apple 64gb azul tela 6,1 câ...",3134.05,https://www.google.com/url?url=https://www.car...
4,"iphone 12 64gb roxo tela 6,1 4g câmera traseir...",3349.00,https://www.google.com/url?url=https://www.car...
5,usado: iphone 12 64gb branco bom - trocafone,3109.00,https://www.google.com/url?url=https://www.bus...
6,iphone 12 64gb tela 6.1 sem carregador e fone ...,3011.00,https://www.google.com/url?url=https://www.bus...
7,vitrine apple iphone 12 64gb verde original - ...,3199.00,https://www.google.com/url?url=https://produto...
8,vitrine iphone 12 preto 64gb,3199.99,https://www.google.com/url?url=https://www.car...
9,vitrine apple iphone 12 64gb original - verde,3199.00,https://www.google.com/url?url=https://produto...


Exportar a base de ofertas para Excel

In [6]:
#exportar pro excel
tabela_ofertas.to_excel("OfertasI.xlsx",index=False)

Enviando o e-mail

In [7]:
len(tabela_ofertas)

59

In [8]:
# enviar por e-mail o resultado da tabela
import win32com.client as win32

#verificando se existe alguma oferta dentro da tabela de ofertas
if len(tabela_ofertas) > 0:
    # vou enviar email
    outlook = win32.Dispatch('outlook.application')
    mail = outlook.CreateItem(0)
    mail.To = 'alvesvictor125@gmail.com'
    mail.Subject = 'Produto(s) Encontrado(s) na faixa de preço desejada'
    mail.HTMLBody = f"""
    <p>Prezados,</p>
    <p>Encontramos alguns produtos em oferta dentro da faixa de preço desejada. Segue tabela com detalhes</p>
    {tabela_ofertas.to_html(index=False)}
    <p>Qualquer dúvida estou à disposição</p>
    <p>Att.,</p>
    """
    
    mail.Send()

driver.quit()  

com_error: (-2147221005, 'Cadeia de caracteres de classe inválida', None, None)